#### Setup:

In [ ]:
!pip install -U tensorflow keras

In [ ]:
# Import general Python libraries
import pandas as pd
import numpy as np
import random
import sklearn
import seaborn as sns
import os
import matplotlib.pyplot as plt
import itertools

In [ ]:
# Specify seeds for random-operations
seed_value = 0
os.environ['PYTHONHASHSEED']=str(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

In [ ]:
# Import sklearn-specific modules
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer, ENGLISH_STOP_WORDS

In [ ]:
# Import tensorflow-specific modules
import tensorflow as tf
tf.random.set_seed(seed_value)
print("Tensorflow Version: {}".format(tf.__version__))
print("Keras Version: {}".format(tf.keras.__version__))

Tensorflow Version: 2.4.1
Keras Version: 2.4.0


In [ ]:
# Import keras-specific modules
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adadelta, RMSprop, Adam, Adamax, Nadam
from tensorflow.keras.regularizers import L1, L2
from tensorflow.keras.initializers import GlorotNormal, GlorotUniform, HeNormal, HeUniform, LecunNormal, LecunUniform
from tensorflow.keras.metrics import AUC, Precision, Recall

In [ ]:
# Set pandas options
pd.set_option("display.max_columns", None)

In [ ]:
# Adjust stopword list
stopwords = list(ENGLISH_STOP_WORDS) + ["kickstarter", "year", "dollar"]
len(stopwords)

321

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### a) Prepare Dataset

In [ ]:
# Import Dataset
kickstarter_df = pd.read_csv("04_Final Datasets/Kickstarter_Text.csv", index_col=0)
print(kickstarter_df.shape)
print(len(kickstarter_df.index.unique()))
kickstarter_df.head(1)

(246891, 7)
246891


,campaign_successful,title,blurb,story,risks,reward_description,creator_bio
22821161,0,sentio golf putters. feel is the difference,choose the feel you want with our patented flo...,sentio putters feature a unique floating face...,high tech process although we have made severa...,our eternal gratitude. every little bit helps ...,sentio golf is driven to produce the most adva...


In [ ]:
# Convert dataset and target variable to Numpy Arrays
y = kickstarter_df["campaign_successful"].to_numpy()
text = kickstarter_df["story"].to_numpy()

print(type(y))
print(y.shape)
print(type(text))
print(text.shape)

<class 'numpy.ndarray'>
(246891,)
<class 'numpy.ndarray'>
(246891,)


In [ ]:
# Split dataset into training, subtraining, validation, and test set
train_size = round(kickstarter_df.shape[0]*0.7*1)
val_size = round(kickstarter_df.shape[0]*0.15*1)
test_size = round(kickstarter_df.shape[0]*1) - val_size - train_size

text_train, text_test, y_train, y_test = train_test_split(text, y, train_size=(train_size+val_size),test_size=test_size, shuffle=True, stratify=y, random_state=seed_value)
text_subtrain, text_val, y_subtrain, y_val = train_test_split(text_train, y_train, train_size=train_size, test_size=val_size, shuffle=True, stratify=y_train, random_state=seed_value)

print("Shape of X_train: {}".format(text_train.shape))
print("Shape of y_train: {}".format(y_train.shape))
print("Shape of X_subtrain: {}".format(text_subtrain.shape))
print("Shape of y_subtrain: {}".format(y_subtrain.shape))
print("Shape of X_val: {}".format(text_val.shape))
print("Shape of y_val: {}".format(y_val.shape))
print("Shape of X_test: {}".format(text_test.shape))
print("Shape of y_test: {}".format(y_test.shape))

Shape of X_train: (209858,)
Shape of y_train: (209858,)
Shape of X_subtrain: (172824,)
Shape of y_subtrain: (172824,)
Shape of X_val: (37034,)
Shape of y_val: (37034,)
Shape of X_test: (37033,)
Shape of y_test: (37033,)


#### b) Baseline

This model provides the baseline, which will be used as starting point for the hyperparameter search. The goal is to gradually improve the model performance.

**Hyperparameters:**
- CountVectorizer; No Limits; English Stop Words; No N-grams
- Batch_Size = 512
- No Regularization
- Weight Initializer = GlorotNormal
- Optimizer = RMSProp_centered

In [ ]:
# Convert Text into a BOW representation
vectorizer = CountVectorizer(stop_words=stopwords, ngram_range=(1,1), analyzer="word", min_df=0.0, max_df=1.0, binary=False, dtype=np.float32)
vectorizer.fit(text_subtrain)
X_subtrain = vectorizer.transform(text_subtrain)
X_val = vectorizer.transform(text_val)

print("Shape of X_subtrain: {}".format(X_subtrain.shape))
print("Datatype of X_subtrain: {}, {}".format(type(X_subtrain), X_subtrain.dtype))
print("Shape of X_val: {}".format(X_val.shape))
print("Datatype of X_val: {}, {}".format(type(X_val), X_val.dtype))

Shape of X_subtrain: (172824, 431211)
Datatype of X_subtrain: <class 'scipy.sparse.csr.csr_matrix'>, float32
Shape of X_val: (37034, 431211)
Datatype of X_val: <class 'scipy.sparse.csr.csr_matrix'>, float32


In [ ]:
model = Sequential()
model.add(Input(shape=(X_subtrain.shape[1],), sparse=True, batch_size=512))
model.add(Dense(1, activation="sigmoid", kernel_initializer=GlorotNormal(seed=seed_value)))
model.compile(optimizer=RMSprop(centered=True), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, verbose=0, mode='max', restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 5s 12ms/step - loss: 0.5783 - binary_accuracy: 0.7375 - val_loss: 0.5278 - val_binary_accuracy: 0.7544
Epoch 2/100
338/338 [==============================] - 4s 11ms/step - loss: 0.4882 - binary_accuracy: 0.7795 - val_loss: 0.5269 - val_binary_accuracy: 0.7580
Epoch 3/100
338/338 [==============================] - 4s 11ms/step - loss: 0.4587 - binary_accuracy: 0.7940 - val_loss: 0.5331 - val_binary_accuracy: 0.7555
Epoch 4/100
338/338 [==============================] - 4s 11ms/step - loss: 0.4422 - binary_accuracy: 0.8040 - val_loss: 0.5429 - val_binary_accuracy: 0.7551
Epoch 5/100
338/338 [==============================] - 4s 11ms/step - loss: 0.4296 - binary_accuracy: 0.8104 - val_loss: 0.5508 - val_binary_accuracy: 0.7516
Epoch 6/100
338/338 [==============================] - 4s 11ms/step - loss: 0.4189 - binary_accuracy: 0.8165 - val_loss: 0.5599 - val_binary_accuracy: 0.7517
Epoch 7/100
338/338 [==============================]

Result: model starts to overfit very quickly

#### c) Hyperparameter Tuning

##### Increase the patience to see if the model maybe was stuck in a plateau:

In [ ]:
model = Sequential()
model.add(Input(shape=(X_subtrain.shape[1],), sparse=True, batch_size=512))
model.add(Dense(1, activation="sigmoid", kernel_initializer=GlorotNormal(seed=seed_value)))
model.compile(optimizer=RMSprop(centered=True), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=10, verbose=0, mode='max', restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 5s 12ms/step - loss: 0.5807 - binary_accuracy: 0.7378 - val_loss: 0.5289 - val_binary_accuracy: 0.7570
Epoch 2/100
338/338 [==============================] - 4s 11ms/step - loss: 0.4884 - binary_accuracy: 0.7800 - val_loss: 0.5274 - val_binary_accuracy: 0.7567
Epoch 3/100
338/338 [==============================] - 4s 11ms/step - loss: 0.4592 - binary_accuracy: 0.7935 - val_loss: 0.5353 - val_binary_accuracy: 0.7544
Epoch 4/100
338/338 [==============================] - 4s 11ms/step - loss: 0.4427 - binary_accuracy: 0.8023 - val_loss: 0.5426 - val_binary_accuracy: 0.7544
Epoch 5/100
338/338 [==============================] - 4s 11ms/step - loss: 0.4310 - binary_accuracy: 0.8098 - val_loss: 0.5509 - val_binary_accuracy: 0.7520
Epoch 6/100
338/338 [==============================] - 4s 11ms/step - loss: 0.4162 - binary_accuracy: 0.8168 - val_loss: 0.5596 - val_binary_accuracy: 0.7521
Epoch 7/100
338/338 [==============================]

Result: model was not stuck in a plateau

##### Tune different regularizers:

In [ ]:
counter=0
for regularizer, reg_rate in itertools.product(["L1", "L2"],[0.001, 0.01, 0.1, 0, 1, 10, 100]):
  counter += 1
  print("Round: {}, Params: {} {}".format(counter, regularizer, reg_rate))
  if regularizer=="L1": reg = L1(l1=reg_rate)
  if regularizer=="L2": reg = L2(l2=reg_rate)
  model = Sequential()
  model.add(Input(shape=(X_subtrain.shape[1],), sparse=True, batch_size=512))
  model.add(Dense(1, activation="sigmoid", kernel_regularizer=reg, kernel_initializer=GlorotNormal(seed=seed_value)))
  model.compile(optimizer=RMSprop(centered=True), loss="binary_crossentropy", metrics=["binary_accuracy"])
  history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, verbose=0, mode='max', restore_best_weights=True)])
  print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
  print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Round: 1, Params: L1 0.001
Epoch 1/100
338/338 [==============================] - 5s 14ms/step - loss: 0.8726 - binary_accuracy: 0.7245 - val_loss: 0.8060 - val_binary_accuracy: 0.7384
Epoch 2/100
338/338 [==============================] - 5s 13ms/step - loss: 0.7942 - binary_accuracy: 0.7371 - val_loss: 0.7994 - val_binary_accuracy: 0.7313
Epoch 3/100
338/338 [==============================] - 5s 13ms/step - loss: 0.7913 - binary_accuracy: 0.7363 - val_loss: 0.7958 - val_binary_accuracy: 0.7314
Epoch 4/100
338/338 [==============================] - 5s 13ms/step - loss: 0.7909 - binary_accuracy: 0.7356 - val_loss: 0.7958 - val_binary_accuracy: 0.7343
Epoch 5/100
338/338 [==============================] - 5s 13ms/step - loss: 0.7934 - binary_accuracy: 0.7327 - val_loss: 0.7962 - val_binary_accuracy: 0.7311
Epoch 6/100
338/338 [==============================] - 5s 13ms/step - loss: 0.7905 - binary_accuracy: 0.7347 - val_loss: 0.7966 - val_binary_accuracy: 0.7315
Epoch 7/100
338/338 [====

Result: 
- only very low regularization didn't lead to a decrease in performance
- L2 regularization performed better than L1
- in general: regularization didn't help to decrease overfitting, but rather decreased both training and validation accuracy (probably the dimensionality is just too high)

##### Tune different optimizers:

In [ ]:
counter=0
for optimizer in ["RMSprop", "RMSprop_centered", "Adam", "Adam_amsgrad", "Adamax", "Nadam"]:
  counter += 1
  print("Round: {}, Params: {}".format(counter, optimizer))
  if optimizer=="RMSprop": opt = RMSprop()
  if optimizer=="RMSprop_centered": opt = RMSprop(centered=True)
  if optimizer=="Adam": opt = Adam()
  if optimizer=="Adam_amsgrad": opt = Adam(amsgrad=True)
  if optimizer=="Adamax": opt = Adamax()
  if optimizer=="Nadam": opt = Nadam()
  model = Sequential()
  model.add(Input(shape=(X_subtrain.shape[1],), sparse=True, batch_size=512))
  model.add(Dense(1, activation="sigmoid", kernel_initializer=GlorotNormal(seed=seed_value)))
  model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["binary_accuracy"])
  history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, verbose=0, mode='max', restore_best_weights=True)])
  print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
  print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Round: 1, Params: RMSprop
Epoch 1/100
338/338 [==============================] - 4s 11ms/step - loss: 0.5885 - binary_accuracy: 0.7366 - val_loss: 0.5387 - val_binary_accuracy: 0.7597
Epoch 2/100
338/338 [==============================] - 4s 11ms/step - loss: 0.4974 - binary_accuracy: 0.7833 - val_loss: 0.5311 - val_binary_accuracy: 0.7635
Epoch 3/100
338/338 [==============================] - 4s 10ms/step - loss: 0.4676 - binary_accuracy: 0.7929 - val_loss: 0.5336 - val_binary_accuracy: 0.7573
Epoch 4/100
338/338 [==============================] - 4s 11ms/step - loss: 0.4447 - binary_accuracy: 0.8039 - val_loss: 0.5395 - val_binary_accuracy: 0.7566
Epoch 5/100
338/338 [==============================] - 4s 11ms/step - loss: 0.4310 - binary_accuracy: 0.8110 - val_loss: 0.5477 - val_binary_accuracy: 0.7561
Epoch 6/100
338/338 [==============================] - 4s 10ms/step - loss: 0.4208 - binary_accuracy: 0.8145 - val_loss: 0.5553 - val_binary_accuracy: 0.7537
Epoch 7/100
338/338 [=====

Result:
- RMSprop led to the best result and the least overfitting (i.e. use RMSprop)

##### Tune different weight initializers:

In [ ]:
counter=0
for initializer in ["glorot_normal", "glorot_uniform", "he_normal", "he_uniform", "lecun_normal", "lecun_uniform"]:
  counter += 1
  print("Round: {}, Params: {}".format(counter, initializer))
  if initializer=="glorot_normal": init = GlorotNormal(seed=seed_value)
  if initializer=="glorot_uniform": init = GlorotUniform(seed=seed_value)
  if initializer=="he_normal": init = HeNormal(seed=seed_value)
  if initializer=="he_uniform": init = HeUniform(seed=seed_value)
  if initializer=="lecun_normal": init = LecunNormal(seed=seed_value)
  if initializer=="lecun_uniform": init = LecunUniform(seed=seed_value)
  model = Sequential()
  model.add(Input(shape=(X_subtrain.shape[1],), sparse=True, batch_size=512))
  model.add(Dense(1, activation="sigmoid", kernel_initializer=init))
  model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])
  history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, verbose=0, mode='max', restore_best_weights=True)])
  print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
  print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Round: 1, Params: glorot_normal
Epoch 1/100
338/338 [==============================] - 4s 11ms/step - loss: 0.5872 - binary_accuracy: 0.7365 - val_loss: 0.5372 - val_binary_accuracy: 0.7613
Epoch 2/100
338/338 [==============================] - 4s 10ms/step - loss: 0.4971 - binary_accuracy: 0.7835 - val_loss: 0.5312 - val_binary_accuracy: 0.7601
Epoch 3/100
338/338 [==============================] - 4s 11ms/step - loss: 0.4662 - binary_accuracy: 0.7946 - val_loss: 0.5336 - val_binary_accuracy: 0.7591
Epoch 4/100
338/338 [==============================] - 4s 10ms/step - loss: 0.4505 - binary_accuracy: 0.8012 - val_loss: 0.5440 - val_binary_accuracy: 0.7547
Epoch 5/100
338/338 [==============================] - 4s 10ms/step - loss: 0.4326 - binary_accuracy: 0.8091 - val_loss: 0.5470 - val_binary_accuracy: 0.7538
Epoch 6/100
338/338 [==============================] - 4s 10ms/step - loss: 0.4212 - binary_accuracy: 0.8154 - val_loss: 0.5555 - val_binary_accuracy: 0.7546
Training Accuracy: 0

Result:
- normal distributions worked a little bit better
- He was the best; Lecun the second-best
- however, the differences were only marginal

##### Tune different batch sizes:

In [ ]:
counter=0
for batch_size in [16, 32, 64, 128, 256, 512, 1024]:
  counter += 1
  print("Round: {}, Params: {}".format(counter, batch_size))
  model = Sequential()
  model.add(Input(shape=(X_subtrain.shape[1],), sparse=True, batch_size=batch_size))
  model.add(Dense(1, activation="sigmoid", kernel_initializer=HeNormal(seed=seed_value)))
  model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])
  history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=batch_size, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, verbose=0, mode='max', restore_best_weights=True)])
  print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=batch_size, verbose=0)[1]))
  print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=batch_size, verbose=0)[1]))

Round: 1, Params: 16
Epoch 1/100
10802/10802 [==============================] - 58s 5ms/step - loss: 0.6057 - binary_accuracy: 0.7413 - val_loss: 0.7111 - val_binary_accuracy: 0.7435
Epoch 2/100
10802/10802 [==============================] - 56s 5ms/step - loss: 0.6710 - binary_accuracy: 0.7585 - val_loss: 0.8250 - val_binary_accuracy: 0.7382
Epoch 3/100
10802/10802 [==============================] - 56s 5ms/step - loss: 0.7363 - binary_accuracy: 0.7580 - val_loss: 0.9188 - val_binary_accuracy: 0.7293
Epoch 4/100
10802/10802 [==============================] - 56s 5ms/step - loss: 0.7778 - binary_accuracy: 0.7613 - val_loss: 1.0014 - val_binary_accuracy: 0.7278
Epoch 5/100
10802/10802 [==============================] - 56s 5ms/step - loss: 0.8094 - binary_accuracy: 0.7650 - val_loss: 1.0754 - val_binary_accuracy: 0.7265
Epoch 6/100
10802/10802 [==============================] - 55s 5ms/step - loss: 0.8435 - binary_accuracy: 0.7680 - val_loss: 1.1393 - val_binary_accuracy: 0.7215
Trainin

Result: higher batch sizes led to the best results

##### Test different max_df variations:

In [ ]:
vectorizer = CountVectorizer(stop_words=stopwords, ngram_range=(1,1), analyzer="word", min_df=0.0, max_df=0.0001, binary=False, dtype=np.float32)
vectorizer.fit(text_subtrain)
X_subtrain = vectorizer.transform(text_subtrain)
X_val = vectorizer.transform(text_val)
print("Tokens: {}".format(len(vectorizer.vocabulary_)))

model = Sequential()
model.add(Input(shape=(X_subtrain.shape[1],), sparse=True, batch_size=512))
model.add(Dense(1, activation="sigmoid", kernel_initializer=HeNormal(seed=seed_value)))
model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, verbose=0, mode='max', restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Tokens: 388183
Epoch 1/100
338/338 [==============================] - 3s 8ms/step - loss: 0.6803 - binary_accuracy: 0.6478 - val_loss: 0.6537 - val_binary_accuracy: 0.6511
Epoch 2/100
338/338 [==============================] - 3s 8ms/step - loss: 0.6381 - binary_accuracy: 0.6544 - val_loss: 0.6420 - val_binary_accuracy: 0.6513
Epoch 3/100
338/338 [==============================] - 3s 8ms/step - loss: 0.6207 - binary_accuracy: 0.6558 - val_loss: 0.6397 - val_binary_accuracy: 0.6522
Epoch 4/100
338/338 [==============================] - 3s 8ms/step - loss: 0.6081 - binary_accuracy: 0.6640 - val_loss: 0.6380 - val_binary_accuracy: 0.6537
Epoch 5/100
338/338 [==============================] - 3s 8ms/step - loss: 0.5973 - binary_accuracy: 0.6716 - val_loss: 0.6366 - val_binary_accuracy: 0.6557
Epoch 6/100
338/338 [==============================] - 3s 8ms/step - loss: 0.5891 - binary_accuracy: 0.6771 - val_loss: 0.6356 - val_binary_accuracy: 0.6574
Epoch 7/100
338/338 [======================

Results:
- max_df=0.01 -> nearly no tokens were dropped -> reduce
- max_df=0.0001 -> reduced the number of tokens to some extent, but the accuracy dropped significantly
- i.e. tuning max_df is the wrong approach

##### Tune different min_df variations:

In [ ]:
vectorizer = CountVectorizer(stop_words=stopwords, ngram_range=(1,1), analyzer="word", min_df=0.0001, max_df=1.0, binary=False, dtype=np.float32)
vectorizer.fit(text_subtrain)
X_subtrain = vectorizer.transform(text_subtrain)
X_val = vectorizer.transform(text_val)
print("Tokens: {}".format(len(vectorizer.vocabulary_)))

model = Sequential()
model.add(Input(shape=(X_subtrain.shape[1],), sparse=True, batch_size=512))
model.add(Dense(1, activation="sigmoid", kernel_initializer=HeNormal(seed=seed_value)))
model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, verbose=0, mode='max', restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Tokens: 43028
Epoch 1/100
338/338 [==============================] - 3s 7ms/step - loss: 0.5901 - binary_accuracy: 0.7316 - val_loss: 0.5390 - val_binary_accuracy: 0.7637
Epoch 2/100
338/338 [==============================] - 2s 6ms/step - loss: 0.5048 - binary_accuracy: 0.7793 - val_loss: 0.5331 - val_binary_accuracy: 0.7627
Epoch 3/100
338/338 [==============================] - 2s 6ms/step - loss: 0.4791 - binary_accuracy: 0.7884 - val_loss: 0.5346 - val_binary_accuracy: 0.7598
Epoch 4/100
338/338 [==============================] - 2s 6ms/step - loss: 0.4674 - binary_accuracy: 0.7916 - val_loss: 0.5429 - val_binary_accuracy: 0.7549
Epoch 5/100
338/338 [==============================] - 2s 6ms/step - loss: 0.4555 - binary_accuracy: 0.7962 - val_loss: 0.5478 - val_binary_accuracy: 0.7554
Epoch 6/100
338/338 [==============================] - 2s 6ms/step - loss: 0.4503 - binary_accuracy: 0.7997 - val_loss: 0.5559 - val_binary_accuracy: 0.7525
Training Accuracy: 0.783
Validation Accuracy

Results:
- min_df=0.0001 (17 documents) -> 43,000 Token -> 76.4% accuracy (a little bit more overfitting)
- min_df=0.001 (172 documents) -> 13,600 Token -> 76.1% accuracy (way less overfitting)
- min_df=0.01 (1720 documents) -> 3,000 Token -> 75.1% accuracy (nearly no overfitting)
- min_df=0.1 (17,200 documents) -> 200 Token -> 71% accuracy
- i.e. tune model to have values in the range of 10,000 - 50,000 token


##### Tune different max_feature values:

In [ ]:
vectorizer = CountVectorizer(stop_words=stopwords, ngram_range=(1,1), analyzer="word", min_df=0.0, max_df=1.0, max_features=45000, binary=False, dtype=np.float32)
vectorizer.fit(text_subtrain)
X_subtrain = vectorizer.transform(text_subtrain)
X_val = vectorizer.transform(text_val)
print("Tokens: {}".format(len(vectorizer.vocabulary_)))

model = Sequential()
model.add(Input(shape=(X_subtrain.shape[1],), sparse=True, batch_size=512))
model.add(Dense(1, activation="sigmoid", kernel_initializer=HeNormal(seed=seed_value)))
model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, verbose=0, mode='max', restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Tokens: 45000
Epoch 1/100
338/338 [==============================] - 3s 7ms/step - loss: 0.5901 - binary_accuracy: 0.7325 - val_loss: 0.5379 - val_binary_accuracy: 0.7632
Epoch 2/100
338/338 [==============================] - 2s 7ms/step - loss: 0.5069 - binary_accuracy: 0.7783 - val_loss: 0.5316 - val_binary_accuracy: 0.7621
Epoch 3/100
338/338 [==============================] - 2s 7ms/step - loss: 0.4789 - binary_accuracy: 0.7871 - val_loss: 0.5344 - val_binary_accuracy: 0.7604
Epoch 4/100
338/338 [==============================] - 2s 7ms/step - loss: 0.4634 - binary_accuracy: 0.7920 - val_loss: 0.5401 - val_binary_accuracy: 0.7563
Epoch 5/100
338/338 [==============================] - 3s 7ms/step - loss: 0.4541 - binary_accuracy: 0.7969 - val_loss: 0.5477 - val_binary_accuracy: 0.7543
Epoch 6/100
338/338 [==============================] - 2s 7ms/step - loss: 0.4472 - binary_accuracy: 0.7995 - val_loss: 0.5545 - val_binary_accuracy: 0.7533
Training Accuracy: 0.782
Validation Accuracy

Results:
- max_features=10,000 -> 77.7% vs. 76%
- max_features=20,000 -> 77.8% vs. 76.1%
- max_features=30,000 -> 78.1% vs. 76.3%
- max_features=35,000 -> 78.2% vs. 76.4%
- max_features=40,000 -> 78.4% vs. 76.4%
- max_features=45,000 -> 78.2% vs. 76.3%
- max_features=50,000 -> 78.4% vs. 76.3%

##### See if different types of vectorizers help:

In [ ]:
# Binary vectorization instead of count-based
vectorizer = CountVectorizer(stop_words=stopwords, ngram_range=(1,1), analyzer="word", min_df=0.0, max_df=1.0, max_features=35000, binary=True, dtype=np.float32)
vectorizer.fit(text_subtrain)
X_subtrain = vectorizer.transform(text_subtrain)
X_val = vectorizer.transform(text_val)
print("Tokens: {}".format(len(vectorizer.vocabulary_)))

model = Sequential()
model.add(Input(shape=(X_subtrain.shape[1],), sparse=True, batch_size=512))
model.add(Dense(1, activation="sigmoid", kernel_initializer=HeNormal(seed=seed_value)))
model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, verbose=0, mode='max', restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Tokens: 35000
Epoch 1/100
338/338 [==============================] - 3s 8ms/step - loss: 0.5858 - binary_accuracy: 0.7266 - val_loss: 0.5192 - val_binary_accuracy: 0.7588
Epoch 2/100
338/338 [==============================] - 2s 6ms/step - loss: 0.4949 - binary_accuracy: 0.7764 - val_loss: 0.5061 - val_binary_accuracy: 0.7665
Epoch 3/100
338/338 [==============================] - 2s 6ms/step - loss: 0.4708 - binary_accuracy: 0.7871 - val_loss: 0.5028 - val_binary_accuracy: 0.7658
Epoch 4/100
338/338 [==============================] - 2s 6ms/step - loss: 0.4560 - binary_accuracy: 0.7943 - val_loss: 0.5032 - val_binary_accuracy: 0.7644
Epoch 5/100
338/338 [==============================] - 2s 6ms/step - loss: 0.4470 - binary_accuracy: 0.7977 - val_loss: 0.5047 - val_binary_accuracy: 0.7633
Epoch 6/100
338/338 [==============================] - 2s 6ms/step - loss: 0.4391 - binary_accuracy: 0.8024 - val_loss: 0.5077 - val_binary_accuracy: 0.7614
Epoch 7/100
338/338 [=======================

In [ ]:
# Tfidf vectorization instead of count-based
vectorizer = TfidfVectorizer(stop_words=stopwords, ngram_range=(1,1), analyzer="word", min_df=0.0, max_df=1.0, max_features=35000, dtype=np.float32)
vectorizer.fit(text_subtrain)
X_subtrain = vectorizer.transform(text_subtrain)
X_val = vectorizer.transform(text_val)
X_subtrain.sort_indices()
X_val.sort_indices()
print("Tokens: {}".format(len(vectorizer.vocabulary_)))

model = Sequential()
model.add(Input(shape=(X_subtrain.shape[1],), sparse=True, batch_size=512))
model.add(Dense(1, activation="sigmoid", kernel_initializer=HeNormal(seed=seed_value)))
model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, verbose=0, mode='max', restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Tokens: 35000
Epoch 1/100
338/338 [==============================] - 3s 7ms/step - loss: 0.6518 - binary_accuracy: 0.6467 - val_loss: 0.5978 - val_binary_accuracy: 0.6762
Epoch 2/100
338/338 [==============================] - 2s 6ms/step - loss: 0.5850 - binary_accuracy: 0.6937 - val_loss: 0.5628 - val_binary_accuracy: 0.7195
Epoch 3/100
338/338 [==============================] - 2s 6ms/step - loss: 0.5518 - binary_accuracy: 0.7307 - val_loss: 0.5423 - val_binary_accuracy: 0.7354
Epoch 4/100
338/338 [==============================] - 2s 6ms/step - loss: 0.5320 - binary_accuracy: 0.7449 - val_loss: 0.5289 - val_binary_accuracy: 0.7422
Epoch 5/100
338/338 [==============================] - 2s 6ms/step - loss: 0.5181 - binary_accuracy: 0.7525 - val_loss: 0.5196 - val_binary_accuracy: 0.7466
Epoch 6/100
338/338 [==============================] - 2s 6ms/step - loss: 0.5091 - binary_accuracy: 0.7542 - val_loss: 0.5128 - val_binary_accuracy: 0.7504
Epoch 7/100
338/338 [=======================

Result: Tfidf-rescaling looks very promising

In [ ]:
# Tfidf vectorization with sub-linear term-frequency instead of count-based
vectorizer = TfidfVectorizer(stop_words=stopwords, ngram_range=(1,1), analyzer="word", min_df=0.0, max_df=1.0, max_features=35000, sublinear_tf=True ,dtype=np.float32)
vectorizer.fit(text_subtrain)
X_subtrain = vectorizer.transform(text_subtrain)
X_val = vectorizer.transform(text_val)
X_subtrain.sort_indices()
X_val.sort_indices()
print("Tokens: {}".format(len(vectorizer.vocabulary_)))

model = Sequential()
model.add(Input(shape=(X_subtrain.shape[1],), sparse=True, batch_size=512))
model.add(Dense(1, activation="sigmoid", kernel_initializer=HeNormal(seed=seed_value)))
model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, verbose=0, mode='max', restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Tokens: 35000
Epoch 1/100
338/338 [==============================] - 3s 7ms/step - loss: 0.6502 - binary_accuracy: 0.6496 - val_loss: 0.5945 - val_binary_accuracy: 0.6848
Epoch 2/100
338/338 [==============================] - 2s 6ms/step - loss: 0.5814 - binary_accuracy: 0.7036 - val_loss: 0.5572 - val_binary_accuracy: 0.7265
Epoch 3/100
338/338 [==============================] - 2s 6ms/step - loss: 0.5455 - binary_accuracy: 0.7387 - val_loss: 0.5354 - val_binary_accuracy: 0.7406
Epoch 4/100
338/338 [==============================] - 2s 6ms/step - loss: 0.5239 - binary_accuracy: 0.7516 - val_loss: 0.5216 - val_binary_accuracy: 0.7477
Epoch 5/100
338/338 [==============================] - 2s 6ms/step - loss: 0.5096 - binary_accuracy: 0.7574 - val_loss: 0.5122 - val_binary_accuracy: 0.7530
Epoch 6/100
338/338 [==============================] - 2s 6ms/step - loss: 0.4983 - binary_accuracy: 0.7634 - val_loss: 0.5054 - val_binary_accuracy: 0.7557
Epoch 7/100
338/338 [=======================

Result: adding sublinear TF helped to increase performance

In [ ]:
# Tfidf vectorization with binary occurence instead of count-based
vectorizer = TfidfVectorizer(stop_words=stopwords, ngram_range=(1,1), analyzer="word", min_df=0.0, max_df=1.0, max_features=35000, binary=True,dtype=np.float32)
vectorizer.fit(text_subtrain)
X_subtrain = vectorizer.transform(text_subtrain)
X_val = vectorizer.transform(text_val)
X_subtrain.sort_indices()
X_val.sort_indices()
print("Tokens: {}".format(len(vectorizer.vocabulary_)))

model = Sequential()
model.add(Input(shape=(X_subtrain.shape[1],), sparse=True, batch_size=512))
model.add(Dense(1, activation="sigmoid", kernel_initializer=HeNormal(seed=seed_value)))
model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, verbose=0, mode='max', restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Tokens: 35000
Epoch 1/100
338/338 [==============================] - 3s 7ms/step - loss: 0.6509 - binary_accuracy: 0.6472 - val_loss: 0.5965 - val_binary_accuracy: 0.6799
Epoch 2/100
338/338 [==============================] - 2s 6ms/step - loss: 0.5835 - binary_accuracy: 0.6985 - val_loss: 0.5592 - val_binary_accuracy: 0.7229
Epoch 3/100
338/338 [==============================] - 2s 6ms/step - loss: 0.5471 - binary_accuracy: 0.7367 - val_loss: 0.5372 - val_binary_accuracy: 0.7378
Epoch 4/100
338/338 [==============================] - 2s 6ms/step - loss: 0.5258 - binary_accuracy: 0.7496 - val_loss: 0.5231 - val_binary_accuracy: 0.7463
Epoch 5/100
338/338 [==============================] - 2s 7ms/step - loss: 0.5119 - binary_accuracy: 0.7566 - val_loss: 0.5133 - val_binary_accuracy: 0.7519
Epoch 6/100
338/338 [==============================] - 2s 7ms/step - loss: 0.5002 - binary_accuracy: 0.7632 - val_loss: 0.5064 - val_binary_accuracy: 0.7542
Epoch 7/100
338/338 [=======================

Result: did not help to improve performance

In [ ]:
# Tfidf vectorization with L1 norm instead of L2
vectorizer = TfidfVectorizer(stop_words=stopwords, ngram_range=(1,1), analyzer="word", min_df=0.0, max_df=1.0, max_features=35000, sublinear_tf=True, norm="l1", dtype=np.float32)
vectorizer.fit(text_subtrain)
X_subtrain = vectorizer.transform(text_subtrain)
X_val = vectorizer.transform(text_val)
X_subtrain.sort_indices()
X_val.sort_indices()
print("Tokens: {}".format(len(vectorizer.vocabulary_)))

model = Sequential()
model.add(Input(shape=(X_subtrain.shape[1],), sparse=True, batch_size=512))
model.add(Dense(1, activation="sigmoid", kernel_initializer=HeNormal(seed=seed_value)))
model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=200, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, verbose=0, mode='max', restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Tokens: 35000
Epoch 1/200
338/338 [==============================] - 3s 7ms/step - loss: 0.6755 - binary_accuracy: 0.6466 - val_loss: 0.6441 - val_binary_accuracy: 0.6501
Epoch 2/200
338/338 [==============================] - 2s 6ms/step - loss: 0.6404 - binary_accuracy: 0.6495 - val_loss: 0.6333 - val_binary_accuracy: 0.6501
Epoch 3/200
338/338 [==============================] - 2s 6ms/step - loss: 0.6309 - binary_accuracy: 0.6503 - val_loss: 0.6264 - val_binary_accuracy: 0.6501
Epoch 4/200
338/338 [==============================] - 2s 6ms/step - loss: 0.6252 - binary_accuracy: 0.6482 - val_loss: 0.6200 - val_binary_accuracy: 0.6501
Epoch 5/200
338/338 [==============================] - 2s 6ms/step - loss: 0.6180 - binary_accuracy: 0.6492 - val_loss: 0.6139 - val_binary_accuracy: 0.6501
Epoch 6/200
338/338 [==============================] - 2s 6ms/step - loss: 0.6110 - binary_accuracy: 0.6511 - val_loss: 0.6083 - val_binary_accuracy: 0.6503
Epoch 7/200
338/338 [=======================

Result: using L1 norm extremely prolonged the training time and did not lead to a better result

Results:
- using Tfidf instead of count-based extremely increased model performance (model did not overfit anymore and was able to learn)
- also adding sublinear_tf helped to further increase model performance

##### Try to see if adding n-grams help:

In [ ]:
# Drop-in replacement with the baseline
vectorizer = CountVectorizer(stop_words=stopwords, ngram_range=(1,2), analyzer="word", min_df=0.0, max_df=1.0, max_features=100000, binary=False, dtype=np.float32)
vectorizer.fit(text_subtrain)
X_subtrain = vectorizer.transform(text_subtrain)
X_val = vectorizer.transform(text_val)
print("Tokens: {}".format(len(vectorizer.vocabulary_)))

model = Sequential()
model.add(Input(shape=(X_subtrain.shape[1],), sparse=True, batch_size=512))
model.add(Dense(1, activation="sigmoid", kernel_initializer=HeNormal(seed=seed_value)))
model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, verbose=0, mode='max', restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Tokens: 100000
Epoch 1/100
338/338 [==============================] - 4s 9ms/step - loss: 0.5803 - binary_accuracy: 0.7420 - val_loss: 0.5271 - val_binary_accuracy: 0.7690
Epoch 2/100
338/338 [==============================] - 3s 8ms/step - loss: 0.4732 - binary_accuracy: 0.7977 - val_loss: 0.5208 - val_binary_accuracy: 0.7682
Epoch 3/100
338/338 [==============================] - 3s 8ms/step - loss: 0.4347 - binary_accuracy: 0.8141 - val_loss: 0.5246 - val_binary_accuracy: 0.7646
Epoch 4/100
338/338 [==============================] - 3s 8ms/step - loss: 0.4068 - binary_accuracy: 0.8267 - val_loss: 0.5328 - val_binary_accuracy: 0.7610
Epoch 5/100
338/338 [==============================] - 3s 8ms/step - loss: 0.3855 - binary_accuracy: 0.8378 - val_loss: 0.5428 - val_binary_accuracy: 0.7603
Epoch 6/100
338/338 [==============================] - 3s 8ms/step - loss: 0.3738 - binary_accuracy: 0.8425 - val_loss: 0.5551 - val_binary_accuracy: 0.7587
Training Accuracy: 0.803
Validation Accurac

Result: definitely helps in increasing accuracy, but still the issue that model overfits very fast -> i.e. switch to Tfidf

In [ ]:
# Tfidf vectorization with sub-linear term-frequency
vectorizer = TfidfVectorizer(stop_words=stopwords, ngram_range=(1,2), analyzer="word", min_df=0.0, max_df=1.0, max_features=100000, sublinear_tf=True, dtype=np.float32)
vectorizer.fit(text_subtrain)
X_subtrain = vectorizer.transform(text_subtrain)
X_val = vectorizer.transform(text_val)
X_subtrain.sort_indices()
X_val.sort_indices()
print("Tokens: {}".format(len(vectorizer.vocabulary_)))

model = Sequential()
model.add(Input(shape=(X_subtrain.shape[1],), sparse=True, batch_size=512))
model.add(Dense(1, activation="sigmoid", kernel_initializer=HeNormal(seed=seed_value)))
model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, verbose=0, mode='max', restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Tokens: 100000
Epoch 1/100
338/338 [==============================] - 7s 18ms/step - loss: 0.6500 - binary_accuracy: 0.6486 - val_loss: 0.5919 - val_binary_accuracy: 0.6902
Epoch 2/100
338/338 [==============================] - 3s 8ms/step - loss: 0.5767 - binary_accuracy: 0.7110 - val_loss: 0.5529 - val_binary_accuracy: 0.7331
Epoch 3/100
338/338 [==============================] - 3s 8ms/step - loss: 0.5379 - binary_accuracy: 0.7471 - val_loss: 0.5304 - val_binary_accuracy: 0.7460
Epoch 4/100
338/338 [==============================] - 3s 8ms/step - loss: 0.5151 - binary_accuracy: 0.7600 - val_loss: 0.5159 - val_binary_accuracy: 0.7535
Epoch 5/100
338/338 [==============================] - 3s 8ms/step - loss: 0.4977 - binary_accuracy: 0.7683 - val_loss: 0.5060 - val_binary_accuracy: 0.7568
Epoch 6/100
338/338 [==============================] - 3s 8ms/step - loss: 0.4861 - binary_accuracy: 0.7732 - val_loss: 0.4987 - val_binary_accuracy: 0.7602
Epoch 7/100
338/338 [=====================

Result: adding n-grams helped to increase model performance

In [ ]:
# Hashing-Vectorizer with 3-grams
vectorizer = HashingVectorizer(stop_words=stopwords, ngram_range=(1,3), analyzer="word", n_features=100000, dtype=np.float32)
vectorizer.fit(text_subtrain)
X_subtrain = vectorizer.transform(text_subtrain)
X_val = vectorizer.transform(text_val)
X_subtrain.sort_indices()
X_val.sort_indices()

model = Sequential()
model.add(Input(shape=(X_subtrain.shape[1],), sparse=True, batch_size=512))
model.add(Dense(1, activation="sigmoid", kernel_initializer=HeUniform(seed=seed_value)))
model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, verbose=0, mode='max', restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 6s 15ms/step - loss: 0.6569 - binary_accuracy: 0.6469 - val_loss: 0.6081 - val_binary_accuracy: 0.6608
Epoch 2/100
338/338 [==============================] - 4s 11ms/step - loss: 0.5943 - binary_accuracy: 0.6792 - val_loss: 0.5744 - val_binary_accuracy: 0.7062
Epoch 3/100
338/338 [==============================] - 4s 12ms/step - loss: 0.5586 - binary_accuracy: 0.7244 - val_loss: 0.5536 - val_binary_accuracy: 0.7271
Epoch 4/100
338/338 [==============================] - 4s 11ms/step - loss: 0.5353 - binary_accuracy: 0.7449 - val_loss: 0.5400 - val_binary_accuracy: 0.7356
Epoch 5/100
338/338 [==============================] - 4s 11ms/step - loss: 0.5189 - binary_accuracy: 0.7546 - val_loss: 0.5302 - val_binary_accuracy: 0.7408
Epoch 6/100
338/338 [==============================] - 4s 12ms/step - loss: 0.5066 - binary_accuracy: 0.7610 - val_loss: 0.5231 - val_binary_accuracy: 0.7451
Epoch 7/100
338/338 [==============================]

Result: adding 3-grams was too much (i.e. stick to 2-grams)

##### Fine-Tune Number of Included 2-Grams:

In [ ]:
# Fine-Tune min_df
vectorizer = TfidfVectorizer(stop_words=stopwords, ngram_range=(1,2), analyzer="word", min_df=0.0001, max_df=1.0, sublinear_tf=True, dtype=np.float32)
vectorizer.fit(text_subtrain)
X_subtrain = vectorizer.transform(text_subtrain)
X_val = vectorizer.transform(text_val)
X_subtrain.sort_indices()
X_val.sort_indices()
print("Tokens: {}".format(len(vectorizer.vocabulary_)))

model = Sequential()
model.add(Input(shape=(X_subtrain.shape[1],), sparse=True, batch_size=512))
model.add(Dense(1, activation="sigmoid", kernel_initializer=HeNormal(seed=seed_value)))
model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, verbose=0, mode='max', restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Tokens: 264282
Epoch 1/100
338/338 [==============================] - 5s 13ms/step - loss: 0.6517 - binary_accuracy: 0.6503 - val_loss: 0.5949 - val_binary_accuracy: 0.6845
Epoch 2/100
338/338 [==============================] - 4s 11ms/step - loss: 0.5781 - binary_accuracy: 0.7063 - val_loss: 0.5565 - val_binary_accuracy: 0.7286
Epoch 3/100
338/338 [==============================] - 4s 12ms/step - loss: 0.5386 - binary_accuracy: 0.7458 - val_loss: 0.5332 - val_binary_accuracy: 0.7461
Epoch 4/100
338/338 [==============================] - 7s 20ms/step - loss: 0.5138 - binary_accuracy: 0.7619 - val_loss: 0.5183 - val_binary_accuracy: 0.7518
Epoch 5/100
338/338 [==============================] - 4s 11ms/step - loss: 0.4952 - binary_accuracy: 0.7697 - val_loss: 0.5078 - val_binary_accuracy: 0.7564
Epoch 6/100
338/338 [==============================] - 4s 11ms/step - loss: 0.4806 - binary_accuracy: 0.7781 - val_loss: 0.5002 - val_binary_accuracy: 0.7605
Epoch 7/100
338/338 [================

Results:
- min_df=0.0 -> 0 documents -> 15,000,000 Token -> too high-dimensional; overfits strongly; extremely slow learning -> 94.1% vs. 78.1%
- min_df=0.1 -> 17,200 documents -> 200 Token -> bad performance -> 72% vs. 71.7%
- min_df=0.01 -> 1,720 documents -> 3000 Token -> better performance, but still potential to increase number of token -> 77% vs. 76.2%
- min_df=0.001 -> 172 documents -> 25,000 Token -> 79.9% vs. 77.4%
- min_df=0.0001 -> 17 documents -> 250,000 Token -> overfitted too much -> 85.6% vs. 77.9%
- optimal number lies somewhere between 50,000 and 200,000 Token

In [ ]:
# Fine-Tune max_features
vectorizer = TfidfVectorizer(stop_words=stopwords, ngram_range=(1,2), analyzer="word", max_features=150000, sublinear_tf=True, dtype=np.float32)
vectorizer.fit(text_subtrain)
X_subtrain = vectorizer.transform(text_subtrain)
X_val = vectorizer.transform(text_val)
X_subtrain.sort_indices()
X_val.sort_indices()
print("Tokens: {}".format(len(vectorizer.vocabulary_)))

model = Sequential()
model.add(Input(shape=(X_subtrain.shape[1],), sparse=True, batch_size=512))
model.add(Dense(1, activation="sigmoid", kernel_initializer=HeNormal(seed=seed_value)))
model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, verbose=0, mode='max', restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Tokens: 150000
Epoch 1/100
338/338 [==============================] - 8s 21ms/step - loss: 0.6509 - binary_accuracy: 0.6494 - val_loss: 0.5930 - val_binary_accuracy: 0.6863
Epoch 2/100
338/338 [==============================] - 4s 10ms/step - loss: 0.5771 - binary_accuracy: 0.7072 - val_loss: 0.5542 - val_binary_accuracy: 0.7318
Epoch 3/100
338/338 [==============================] - 4s 10ms/step - loss: 0.5382 - binary_accuracy: 0.7468 - val_loss: 0.5313 - val_binary_accuracy: 0.7459
Epoch 4/100
338/338 [==============================] - 4s 10ms/step - loss: 0.5133 - binary_accuracy: 0.7608 - val_loss: 0.5167 - val_binary_accuracy: 0.7524
Epoch 5/100
338/338 [==============================] - 4s 10ms/step - loss: 0.4959 - binary_accuracy: 0.7698 - val_loss: 0.5065 - val_binary_accuracy: 0.7570
Epoch 6/100
338/338 [==============================] - 4s 10ms/step - loss: 0.4837 - binary_accuracy: 0.7759 - val_loss: 0.4991 - val_binary_accuracy: 0.7608
Epoch 7/100
338/338 [================

Results:
- max_features = 25,000 -> 79.4% vs. 77.3%
- max_features = 50,000 -> 80.9% vs. 77.4%
- max_features = 75.000 -> 82.2% vs 77.6%
- max_features = 100,000 -> 82.9% vs. 77.7%
- max_features = 125,000 -> 82.8% vs. 77.7%
- max_features = 150,000 -> 84% vs. 77.8%
- i.e. max_features = 75,000 provides the best trade-off between good accuracy and low overfitting

In [ ]:
# Perform final data transformation based on the best found model
vectorizer = TfidfVectorizer(stop_words=stopwords, ngram_range=(1,2), analyzer="word", max_features=75000, sublinear_tf=True, dtype=np.float32)
vectorizer.fit(text_subtrain)
X_subtrain = vectorizer.transform(text_subtrain)
X_val = vectorizer.transform(text_val)
X_subtrain.sort_indices()
X_val.sort_indices()
print("Tokens: {}".format(len(vectorizer.vocabulary_)))

Tokens: 75000


In [ ]:
model = Sequential()
model.add(Input(shape=(X_subtrain.shape[1],), sparse=True, batch_size=512))
model.add(Dense(1, activation="sigmoid", kernel_initializer=HeNormal(seed=seed_value)))
model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, verbose=0, mode='max', restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 4s 9ms/step - loss: 0.6495 - binary_accuracy: 0.6504 - val_loss: 0.5911 - val_binary_accuracy: 0.6901
Epoch 2/100
338/338 [==============================] - 3s 8ms/step - loss: 0.5750 - binary_accuracy: 0.7120 - val_loss: 0.5524 - val_binary_accuracy: 0.7313
Epoch 3/100
338/338 [==============================] - 3s 8ms/step - loss: 0.5376 - binary_accuracy: 0.7462 - val_loss: 0.5300 - val_binary_accuracy: 0.7467
Epoch 4/100
338/338 [==============================] - 3s 8ms/step - loss: 0.5163 - binary_accuracy: 0.7583 - val_loss: 0.5157 - val_binary_accuracy: 0.7511
Epoch 5/100
338/338 [==============================] - 3s 8ms/step - loss: 0.5003 - binary_accuracy: 0.7660 - val_loss: 0.5059 - val_binary_accuracy: 0.7561
Epoch 6/100
338/338 [==============================] - 3s 8ms/step - loss: 0.4881 - binary_accuracy: 0.7723 - val_loss: 0.4988 - val_binary_accuracy: 0.7602
Epoch 7/100
338/338 [==============================] - 3s 

##### Fine-Tune Logistic Regression Based On Tfidf-Dataset with 2-Grams

In [ ]:
# Fine-Tune Regularizers
counter=0
for regularizer, reg_rate in itertools.product(["L1", "L2"],[0.0001, 0.001, 0.01, 0.1, 0, 1, 10]):
  counter += 1
  print("Round: {}, Params: {} {}".format(counter, regularizer, reg_rate))
  if regularizer=="L1": reg = L1(l1=reg_rate)
  if regularizer=="L2": reg = L2(l2=reg_rate)
  model = Sequential()
  model.add(Input(shape=(X_subtrain.shape[1],), sparse=True, batch_size=512))
  model.add(Dense(1, activation="sigmoid", kernel_regularizer=reg, kernel_initializer=HeNormal(seed=seed_value)))
  model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])
  history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, verbose=0, mode='max', restore_best_weights=True)])
  print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
  print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Round: 1, Params: L1 0.0001
Epoch 1/100
338/338 [==============================] - 4s 9ms/step - loss: 0.6745 - binary_accuracy: 0.6493 - val_loss: 0.6438 - val_binary_accuracy: 0.6501
Epoch 2/100
338/338 [==============================] - 3s 9ms/step - loss: 0.6380 - binary_accuracy: 0.6506 - val_loss: 0.6268 - val_binary_accuracy: 0.6587
Epoch 3/100
338/338 [==============================] - 6s 18ms/step - loss: 0.6221 - binary_accuracy: 0.6656 - val_loss: 0.6160 - val_binary_accuracy: 0.6802
Epoch 4/100
338/338 [==============================] - 3s 9ms/step - loss: 0.6128 - binary_accuracy: 0.6853 - val_loss: 0.6090 - val_binary_accuracy: 0.6960
Epoch 5/100
338/338 [==============================] - 5s 13ms/step - loss: 0.6056 - binary_accuracy: 0.7008 - val_loss: 0.6041 - val_binary_accuracy: 0.7034
Epoch 6/100
338/338 [==============================] - 3s 9ms/step - loss: 0.6019 - binary_accuracy: 0.7072 - val_loss: 0.6006 - val_binary_accuracy: 0.7101
Epoch 7/100
338/338 [=======

Result: regularization led to a significant drop in model performance (i.e. don't use)

In [ ]:
# Tune different optimizers
counter=0
for optimizer in ["RMSprop", "RMSprop_centered", "Adam", "Adam_amsgrad", "Adamax", "Nadam"]:
  counter += 1
  print("Round: {}, Params: {}".format(counter, optimizer))
  if optimizer=="RMSprop": opt = RMSprop()
  if optimizer=="RMSprop_centered": opt = RMSprop(centered=True)
  if optimizer=="Adam": opt = Adam()
  if optimizer=="Adam_amsgrad": opt = Adam(amsgrad=True)
  if optimizer=="Adamax": opt = Adamax()
  if optimizer=="Nadam": opt = Nadam()
  model = Sequential()
  model.add(Input(shape=(X_subtrain.shape[1],), sparse=True, batch_size=512))
  model.add(Dense(1, activation="sigmoid", kernel_initializer=HeNormal(seed=seed_value)))
  model.compile(optimizer=opt, loss="binary_crossentropy", metrics=["binary_accuracy"])
  history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, verbose=0, mode='max', restore_best_weights=True)])
  print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
  print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Round: 1, Params: RMSprop
Epoch 1/100
338/338 [==============================] - 4s 9ms/step - loss: 0.6491 - binary_accuracy: 0.6503 - val_loss: 0.5911 - val_binary_accuracy: 0.6942
Epoch 2/100
338/338 [==============================] - 3s 8ms/step - loss: 0.5757 - binary_accuracy: 0.7131 - val_loss: 0.5524 - val_binary_accuracy: 0.7325
Epoch 3/100
338/338 [==============================] - 3s 8ms/step - loss: 0.5380 - binary_accuracy: 0.7462 - val_loss: 0.5299 - val_binary_accuracy: 0.7460
Epoch 4/100
338/338 [==============================] - 3s 8ms/step - loss: 0.5146 - binary_accuracy: 0.7595 - val_loss: 0.5156 - val_binary_accuracy: 0.7530
Epoch 5/100
338/338 [==============================] - 3s 8ms/step - loss: 0.4987 - binary_accuracy: 0.7683 - val_loss: 0.5059 - val_binary_accuracy: 0.7565
Epoch 6/100
338/338 [==============================] - 3s 8ms/step - loss: 0.4864 - binary_accuracy: 0.7730 - val_loss: 0.4988 - val_binary_accuracy: 0.7595
Epoch 7/100
338/338 [===========

Result: stick with RMSprop, but differences were marginal

In [ ]:
# Fine-tune different weight initializers
counter=0
for initializer in ["glorot_normal", "glorot_uniform", "he_normal", "he_uniform", "lecun_normal", "lecun_uniform"]:
  counter += 1
  print("Round: {}, Params: {}".format(counter, initializer))
  if initializer=="glorot_normal": init = GlorotNormal(seed=seed_value)
  if initializer=="glorot_uniform": init = GlorotUniform(seed=seed_value)
  if initializer=="he_normal": init = HeNormal(seed=seed_value)
  if initializer=="he_uniform": init = HeUniform(seed=seed_value)
  if initializer=="lecun_normal": init = LecunNormal(seed=seed_value)
  if initializer=="lecun_uniform": init = LecunUniform(seed=seed_value)
  model = Sequential()
  model.add(Input(shape=(X_subtrain.shape[1],), sparse=True, batch_size=512))
  model.add(Dense(1, activation="sigmoid", kernel_initializer=init))
  model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])
  history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, verbose=0, mode='max', restore_best_weights=True)])
  print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
  print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Round: 1, Params: glorot_normal
Epoch 1/100
338/338 [==============================] - 4s 9ms/step - loss: 0.6498 - binary_accuracy: 0.6495 - val_loss: 0.5912 - val_binary_accuracy: 0.6906
Epoch 2/100
338/338 [==============================] - 6s 18ms/step - loss: 0.5764 - binary_accuracy: 0.7113 - val_loss: 0.5523 - val_binary_accuracy: 0.7328
Epoch 3/100
338/338 [==============================] - 3s 8ms/step - loss: 0.5383 - binary_accuracy: 0.7458 - val_loss: 0.5299 - val_binary_accuracy: 0.7456
Epoch 4/100
338/338 [==============================] - 3s 8ms/step - loss: 0.5161 - binary_accuracy: 0.7583 - val_loss: 0.5157 - val_binary_accuracy: 0.7525
Epoch 5/100
338/338 [==============================] - 3s 8ms/step - loss: 0.4998 - binary_accuracy: 0.7666 - val_loss: 0.5059 - val_binary_accuracy: 0.7569
Epoch 6/100
338/338 [==============================] - 3s 8ms/step - loss: 0.4878 - binary_accuracy: 0.7723 - val_loss: 0.4987 - val_binary_accuracy: 0.7599
Epoch 7/100
338/338 [====

Result: Glorot Normal performed the best, but differences were marginal

In [ ]:
# Fine-tune different batch sizes
counter=0
for batch_size in [16, 32, 64, 128, 256, 512, 1024]:
  counter += 1
  print("Round: {}, Params: {}".format(counter, batch_size))
  model = Sequential()
  model.add(Input(shape=(X_subtrain.shape[1],), sparse=True, batch_size=batch_size))
  model.add(Dense(1, activation="sigmoid", kernel_initializer=GlorotNormal(seed=seed_value)))
  model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])
  history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=batch_size, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=5, verbose=0, mode='max', restore_best_weights=True)])
  print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=batch_size, verbose=0)[1]))
  print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=batch_size, verbose=0)[1]))

Round: 1, Params: 16
Epoch 1/100
10802/10802 [==============================] - 23s 2ms/step - loss: 0.5908 - binary_accuracy: 0.6929 - val_loss: 0.5170 - val_binary_accuracy: 0.7513
Epoch 2/100
10802/10802 [==============================] - 23s 2ms/step - loss: 0.5023 - binary_accuracy: 0.7614 - val_loss: 0.5035 - val_binary_accuracy: 0.7614
Epoch 3/100
10802/10802 [==============================] - 23s 2ms/step - loss: 0.4874 - binary_accuracy: 0.7710 - val_loss: 0.5047 - val_binary_accuracy: 0.7635
Epoch 4/100
10802/10802 [==============================] - 23s 2ms/step - loss: 0.4832 - binary_accuracy: 0.7761 - val_loss: 0.5092 - val_binary_accuracy: 0.7629
Epoch 5/100
10802/10802 [==============================] - 24s 2ms/step - loss: 0.4832 - binary_accuracy: 0.7777 - val_loss: 0.5156 - val_binary_accuracy: 0.7622
Epoch 6/100
10802/10802 [==============================] - 24s 2ms/step - loss: 0.4815 - binary_accuracy: 0.7808 - val_loss: 0.5236 - val_binary_accuracy: 0.7612
Epoch 7

Result: batch size of 512 was the best

##### Test If Adding Additional Text Attributes Can Increase Model Performance:

###### 1. Include Title + Blurb

In [ ]:
# Import Dataset
kickstarter_df = pd.read_csv("04_Final Datasets/Kickstarter_Text.csv", index_col=0)
print(kickstarter_df.shape)
print(len(kickstarter_df.index.unique()))
kickstarter_df.head(1)

(246891, 7)
246891


,campaign_successful,title,blurb,story,risks,reward_description,creator_bio
22821161,0,sentio golf putters. feel is the difference,choose the feel you want with our patented flo...,sentio putters feature a unique floating face...,high tech process although we have made severa...,our eternal gratitude. every little bit helps ...,sentio golf is driven to produce the most adva...


In [ ]:
# Merge Story + Title + Blurb together
text_df = kickstarter_df["title"] + " " + kickstarter_df["blurb"] + " " + kickstarter_df["story"]
text_df.head()

22821161    sentio golf putters. feel is the difference ch...
22823613    brainade we intend to create an online platfor...
22835897    the alphabet story children s book a richly il...
22845619    the gears lp from scratch. with you. after a y...
22848517    polynesian adventure the board game polynesian...
dtype: object

In [ ]:
# Convert dataset and target variable to Numpy Arrays
y = kickstarter_df["campaign_successful"].to_numpy()
text = text_df.to_numpy()

print(type(y))
print(y.shape)
print(type(text))
print(text.shape)

<class 'numpy.ndarray'>
(246891,)
<class 'numpy.ndarray'>
(246891,)


In [ ]:
# Split dataset into training, subtraining, validation, and test set
train_size = round(kickstarter_df.shape[0]*0.7*1)
val_size = round(kickstarter_df.shape[0]*0.15*1)
test_size = round(kickstarter_df.shape[0]*1) - val_size - train_size

text_train, text_test, y_train, y_test = train_test_split(text, y, train_size=(train_size+val_size),test_size=test_size, shuffle=True, stratify=y, random_state=seed_value)
text_subtrain, text_val, y_subtrain, y_val = train_test_split(text_train, y_train, train_size=train_size, test_size=val_size, shuffle=True, stratify=y_train, random_state=seed_value)

print("Shape of X_train: {}".format(text_train.shape))
print("Shape of y_train: {}".format(y_train.shape))
print("Shape of X_subtrain: {}".format(text_subtrain.shape))
print("Shape of y_subtrain: {}".format(y_subtrain.shape))
print("Shape of X_val: {}".format(text_val.shape))
print("Shape of y_val: {}".format(y_val.shape))
print("Shape of X_test: {}".format(text_test.shape))
print("Shape of y_test: {}".format(y_test.shape))

Shape of X_train: (209858,)
Shape of y_train: (209858,)
Shape of X_subtrain: (172824,)
Shape of y_subtrain: (172824,)
Shape of X_val: (37034,)
Shape of y_val: (37034,)
Shape of X_test: (37033,)
Shape of y_test: (37033,)


In [ ]:
# Perform best data transformation identified in hyperparameter search
vectorizer = TfidfVectorizer(stop_words=stopwords, ngram_range=(1,2), analyzer="word", max_features=75000, sublinear_tf=True, dtype=np.float32)
vectorizer.fit(text_subtrain)
X_subtrain = vectorizer.transform(text_subtrain)
X_val = vectorizer.transform(text_val)
X_subtrain.sort_indices()
X_val.sort_indices()
print("Tokens: {}".format(len(vectorizer.vocabulary_)))

Tokens: 75000


In [ ]:
# Evaluate on best model identified in hyperparameter search
model = Sequential()
model.add(Input(shape=(X_subtrain.shape[1],), sparse=True, batch_size=512))
model.add(Dense(1, activation="sigmoid", kernel_initializer=GlorotNormal(seed=seed_value)))
model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=5, verbose=0, mode="max", restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 3s 7ms/step - loss: 0.6480 - binary_accuracy: 0.6486 - val_loss: 0.5881 - val_binary_accuracy: 0.6949
Epoch 2/100
338/338 [==============================] - 2s 7ms/step - loss: 0.5723 - binary_accuracy: 0.7157 - val_loss: 0.5488 - val_binary_accuracy: 0.7341
Epoch 3/100
338/338 [==============================] - 2s 7ms/step - loss: 0.5338 - binary_accuracy: 0.7498 - val_loss: 0.5262 - val_binary_accuracy: 0.7478
Epoch 4/100
338/338 [==============================] - 2s 7ms/step - loss: 0.5110 - binary_accuracy: 0.7616 - val_loss: 0.5118 - val_binary_accuracy: 0.7544
Epoch 5/100
338/338 [==============================] - 2s 7ms/step - loss: 0.4970 - binary_accuracy: 0.7668 - val_loss: 0.5021 - val_binary_accuracy: 0.7576
Epoch 6/100
338/338 [==============================] - 2s 7ms/step - loss: 0.4834 - binary_accuracy: 0.7725 - val_loss: 0.4948 - val_binary_accuracy: 0.7616
Epoch 7/100
338/338 [==============================] - 5s 

Result: Including Title + Blurb increased model performance

###### 2. Include Title + Blurb + Risks

In [ ]:
# Import Dataset
kickstarter_df = pd.read_csv("04_Final Datasets/Kickstarter_Text.csv", index_col=0)
print(kickstarter_df.shape)
print(len(kickstarter_df.index.unique()))
kickstarter_df.head(1)

(246891, 7)
246891


,campaign_successful,title,blurb,story,risks,reward_description,creator_bio
22821161,0,sentio golf putters. feel is the difference,choose the feel you want with our patented flo...,sentio putters feature a unique floating face...,high tech process although we have made severa...,our eternal gratitude. every little bit helps ...,sentio golf is driven to produce the most adva...


In [ ]:
# Merge Story + Title + Blurb + Risks together
text_df = kickstarter_df["title"] + " " + kickstarter_df["blurb"] + " " + kickstarter_df["story"]+ " " + kickstarter_df["risks"]
text_df.head()

22821161    sentio golf putters. feel is the difference ch...
22823613    brainade we intend to create an online platfor...
22835897    the alphabet story children s book a richly il...
22845619    the gears lp from scratch. with you. after a y...
22848517    polynesian adventure the board game polynesian...
dtype: object

In [ ]:
# Convert dataset and target variable to Numpy Arrays
y = kickstarter_df["campaign_successful"].to_numpy()
text = text_df.to_numpy()

print(type(y))
print(y.shape)
print(type(text))
print(text.shape)

<class 'numpy.ndarray'>
(246891,)
<class 'numpy.ndarray'>
(246891,)


In [ ]:
# Split dataset into training, subtraining, validation, and test set
train_size = round(kickstarter_df.shape[0]*0.7*1)
val_size = round(kickstarter_df.shape[0]*0.15*1)
test_size = round(kickstarter_df.shape[0]*1) - val_size - train_size

text_train, text_test, y_train, y_test = train_test_split(text, y, train_size=(train_size+val_size),test_size=test_size, shuffle=True, stratify=y, random_state=seed_value)
text_subtrain, text_val, y_subtrain, y_val = train_test_split(text_train, y_train, train_size=train_size, test_size=val_size, shuffle=True, stratify=y_train, random_state=seed_value)

print("Shape of X_train: {}".format(text_train.shape))
print("Shape of y_train: {}".format(y_train.shape))
print("Shape of X_subtrain: {}".format(text_subtrain.shape))
print("Shape of y_subtrain: {}".format(y_subtrain.shape))
print("Shape of X_val: {}".format(text_val.shape))
print("Shape of y_val: {}".format(y_val.shape))
print("Shape of X_test: {}".format(text_test.shape))
print("Shape of y_test: {}".format(y_test.shape))

Shape of X_train: (209858,)
Shape of y_train: (209858,)
Shape of X_subtrain: (172824,)
Shape of y_subtrain: (172824,)
Shape of X_val: (37034,)
Shape of y_val: (37034,)
Shape of X_test: (37033,)
Shape of y_test: (37033,)


In [ ]:
# Perform best data transformation identified in hyperparameter search
vectorizer = TfidfVectorizer(stop_words=stopwords, ngram_range=(1,2), analyzer="word", max_features=75000, sublinear_tf=True, dtype=np.float32)
vectorizer.fit(text_subtrain)
X_subtrain = vectorizer.transform(text_subtrain)
X_val = vectorizer.transform(text_val)
X_subtrain.sort_indices()
X_val.sort_indices()
print("Tokens: {}".format(len(vectorizer.vocabulary_)))

Tokens: 75000


In [ ]:
# Evaluate on best model identified in hyperparameter search
model = Sequential()
model.add(Input(shape=(X_subtrain.shape[1],), sparse=True, batch_size=512))
model.add(Dense(1, activation="sigmoid", kernel_initializer=GlorotNormal(seed=seed_value)))
model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=5, verbose=0, mode="max", restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 4s 9ms/step - loss: 0.6437 - binary_accuracy: 0.6510 - val_loss: 0.5810 - val_binary_accuracy: 0.6990
Epoch 2/100
338/338 [==============================] - 3s 8ms/step - loss: 0.5655 - binary_accuracy: 0.7199 - val_loss: 0.5407 - val_binary_accuracy: 0.7399
Epoch 3/100
338/338 [==============================] - 3s 7ms/step - loss: 0.5254 - binary_accuracy: 0.7530 - val_loss: 0.5180 - val_binary_accuracy: 0.7527
Epoch 4/100
338/338 [==============================] - 3s 8ms/step - loss: 0.5032 - binary_accuracy: 0.7649 - val_loss: 0.5037 - val_binary_accuracy: 0.7596
Epoch 5/100
338/338 [==============================] - 3s 8ms/step - loss: 0.4870 - binary_accuracy: 0.7732 - val_loss: 0.4938 - val_binary_accuracy: 0.7636
Epoch 6/100
338/338 [==============================] - 3s 8ms/step - loss: 0.4741 - binary_accuracy: 0.7798 - val_loss: 0.4865 - val_binary_accuracy: 0.7668
Epoch 7/100
338/338 [==============================] - 3s 

Results: adding Risks further increased accuracy

###### 3. Include Title + Blurb + Risks + Creator Bio

In [ ]:
# Import Dataset
kickstarter_df = pd.read_csv("04_Final Datasets/Kickstarter_Text.csv", index_col=0)
print(kickstarter_df.shape)
print(len(kickstarter_df.index.unique()))
kickstarter_df.head(1)

(246891, 7)
246891


,campaign_successful,title,blurb,story,risks,reward_description,creator_bio
22821161,0,sentio golf putters. feel is the difference,choose the feel you want with our patented flo...,sentio putters feature a unique floating face...,high tech process although we have made severa...,our eternal gratitude. every little bit helps ...,sentio golf is driven to produce the most adva...


In [ ]:
# Merge Story + Title + Blurb + Risks + Creator Bio together
text_df = kickstarter_df["title"] + " " + kickstarter_df["blurb"] + " " + kickstarter_df["story"] + " " + kickstarter_df["risks"] + " " + kickstarter_df["creator_bio"]
text_df.head()

22821161    sentio golf putters. feel is the difference ch...
22823613    brainade we intend to create an online platfor...
22835897    the alphabet story children s book a richly il...
22845619    the gears lp from scratch. with you. after a y...
22848517    polynesian adventure the board game polynesian...
dtype: object

In [ ]:
# Convert dataset and target variable to Numpy Arrays
y = kickstarter_df["campaign_successful"].to_numpy()
text = text_df.to_numpy()

print(type(y))
print(y.shape)
print(type(text))
print(text.shape)

<class 'numpy.ndarray'>
(246891,)
<class 'numpy.ndarray'>
(246891,)


In [ ]:
# Split dataset into training, subtraining, validation, and test set
train_size = round(kickstarter_df.shape[0]*0.7*1)
val_size = round(kickstarter_df.shape[0]*0.15*1)
test_size = round(kickstarter_df.shape[0]*1) - val_size - train_size

text_train, text_test, y_train, y_test = train_test_split(text, y, train_size=(train_size+val_size),test_size=test_size, shuffle=True, stratify=y, random_state=seed_value)
text_subtrain, text_val, y_subtrain, y_val = train_test_split(text_train, y_train, train_size=train_size, test_size=val_size, shuffle=True, stratify=y_train, random_state=seed_value)

print("Shape of X_train: {}".format(text_train.shape))
print("Shape of y_train: {}".format(y_train.shape))
print("Shape of X_subtrain: {}".format(text_subtrain.shape))
print("Shape of y_subtrain: {}".format(y_subtrain.shape))
print("Shape of X_val: {}".format(text_val.shape))
print("Shape of y_val: {}".format(y_val.shape))
print("Shape of X_test: {}".format(text_test.shape))
print("Shape of y_test: {}".format(y_test.shape))

Shape of X_train: (209858,)
Shape of y_train: (209858,)
Shape of X_subtrain: (172824,)
Shape of y_subtrain: (172824,)
Shape of X_val: (37034,)
Shape of y_val: (37034,)
Shape of X_test: (37033,)
Shape of y_test: (37033,)


In [ ]:
# Perform best data transformation identified in hyperparameter search
vectorizer = TfidfVectorizer(stop_words=stopwords, ngram_range=(1,2), analyzer="word", max_features=75000, sublinear_tf=True, dtype=np.float32)
vectorizer.fit(text_subtrain)
X_subtrain = vectorizer.transform(text_subtrain)
X_val = vectorizer.transform(text_val)
X_subtrain.sort_indices()
X_val.sort_indices()
print("Tokens: {}".format(len(vectorizer.vocabulary_)))

Tokens: 75000


In [ ]:
# Evaluate on best model identified in hyperparameter search
model = Sequential()
model.add(Input(shape=(X_subtrain.shape[1],), sparse=True, batch_size=512))
model.add(Dense(1, activation="sigmoid", kernel_initializer=GlorotNormal(seed=seed_value)))
model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=5, verbose=0, mode="max", restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 7s 10ms/step - loss: 0.6386 - binary_accuracy: 0.6528 - val_loss: 0.5710 - val_binary_accuracy: 0.7097
Epoch 2/100
338/338 [==============================] - 3s 8ms/step - loss: 0.5548 - binary_accuracy: 0.7285 - val_loss: 0.5300 - val_binary_accuracy: 0.7461
Epoch 3/100
338/338 [==============================] - 3s 8ms/step - loss: 0.5143 - binary_accuracy: 0.7600 - val_loss: 0.5073 - val_binary_accuracy: 0.7597
Epoch 4/100
338/338 [==============================] - 3s 8ms/step - loss: 0.4922 - binary_accuracy: 0.7715 - val_loss: 0.4930 - val_binary_accuracy: 0.7663
Epoch 5/100
338/338 [==============================] - 3s 8ms/step - loss: 0.4760 - binary_accuracy: 0.7791 - val_loss: 0.4832 - val_binary_accuracy: 0.7705
Epoch 6/100
338/338 [==============================] - 3s 8ms/step - loss: 0.4629 - binary_accuracy: 0.7866 - val_loss: 0.4760 - val_binary_accuracy: 0.7739
Epoch 7/100
338/338 [==============================] - 3s

Result: including creator_bio increased model performance

###### 4. Inlude Title + Blurb + Risks + Creator Bio + Reward Description

In [ ]:
# Import Dataset
kickstarter_df = pd.read_csv("04_Final Datasets/Kickstarter_Text.csv", index_col=0)
print(kickstarter_df.shape)
print(len(kickstarter_df.index.unique()))
kickstarter_df.head(1)

(246891, 7)
246891


,campaign_successful,title,blurb,story,risks,reward_description,creator_bio
22821161,0,sentio golf putters. feel is the difference,choose the feel you want with our patented flo...,sentio putters feature a unique floating face...,high tech process although we have made severa...,our eternal gratitude. every little bit helps ...,sentio golf is driven to produce the most adva...


In [ ]:
# Merge Story + Title + Blurb + Risks + Creator Bio + Reward Description together
text_df = kickstarter_df["title"] + " " + kickstarter_df["blurb"] + " " + kickstarter_df["story"] + " " + kickstarter_df["risks"] + " " + kickstarter_df["creator_bio"] + " " + kickstarter_df["reward_description"]
text_df.head()

22821161    sentio golf putters. feel is the difference ch...
22823613    brainade we intend to create an online platfor...
22835897    the alphabet story children s book a richly il...
22845619    the gears lp from scratch. with you. after a y...
22848517    polynesian adventure the board game polynesian...
dtype: object

In [ ]:
# Convert dataset and target variable to Numpy Arrays
y = kickstarter_df["campaign_successful"].to_numpy()
text = text_df.to_numpy()

print(type(y))
print(y.shape)
print(type(text))
print(text.shape)

<class 'numpy.ndarray'>
(246891,)
<class 'numpy.ndarray'>
(246891,)


In [ ]:
# Split dataset into training, subtraining, validation, and test set
train_size = round(kickstarter_df.shape[0]*0.7*1)
val_size = round(kickstarter_df.shape[0]*0.15*1)
test_size = round(kickstarter_df.shape[0]*1) - val_size - train_size

text_train, text_test, y_train, y_test = train_test_split(text, y, train_size=(train_size+val_size),test_size=test_size, shuffle=True, stratify=y, random_state=seed_value)
text_subtrain, text_val, y_subtrain, y_val = train_test_split(text_train, y_train, train_size=train_size, test_size=val_size, shuffle=True, stratify=y_train, random_state=seed_value)

print("Shape of X_train: {}".format(text_train.shape))
print("Shape of y_train: {}".format(y_train.shape))
print("Shape of X_subtrain: {}".format(text_subtrain.shape))
print("Shape of y_subtrain: {}".format(y_subtrain.shape))
print("Shape of X_val: {}".format(text_val.shape))
print("Shape of y_val: {}".format(y_val.shape))
print("Shape of X_test: {}".format(text_test.shape))
print("Shape of y_test: {}".format(y_test.shape))

Shape of X_train: (209858,)
Shape of y_train: (209858,)
Shape of X_subtrain: (172824,)
Shape of y_subtrain: (172824,)
Shape of X_val: (37034,)
Shape of y_val: (37034,)
Shape of X_test: (37033,)
Shape of y_test: (37033,)


In [ ]:
# Perform best data transformation identified in hyperparameter search
vectorizer = TfidfVectorizer(stop_words=stopwords, ngram_range=(1,2), analyzer="word", max_features=75000, sublinear_tf=True, dtype=np.float32)
vectorizer.fit(text_subtrain)
X_subtrain = vectorizer.transform(text_subtrain)
X_val = vectorizer.transform(text_val)
X_subtrain.sort_indices()
X_val.sort_indices()
print("Tokens: {}".format(len(vectorizer.vocabulary_)))

Tokens: 75000


In [ ]:
# Evaluate on best model identified in hyperparameter search
model = Sequential()
model.add(Input(shape=(X_subtrain.shape[1],), sparse=True, batch_size=512))
model.add(Dense(1, activation="sigmoid", kernel_initializer=GlorotNormal(seed=seed_value)))
model.compile(optimizer=RMSprop(), loss="binary_crossentropy", metrics=["binary_accuracy"])
history = model.fit(X_subtrain, y_subtrain, validation_data=(X_val, y_val), batch_size=512, epochs=100, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_binary_accuracy", patience=5, verbose=0, mode="max", restore_best_weights=True)])
print("Training Accuracy: {:.3f}".format(model.evaluate(X_subtrain, y_subtrain, batch_size=512, verbose=0)[1]))
print("Validation Accuracy: {:.3f}".format(model.evaluate(X_val, y_val, batch_size=512, verbose=0)[1]))

Epoch 1/100
338/338 [==============================] - 9s 11ms/step - loss: 0.6359 - binary_accuracy: 0.6571 - val_loss: 0.5634 - val_binary_accuracy: 0.7211
Epoch 2/100
338/338 [==============================] - 3s 9ms/step - loss: 0.5473 - binary_accuracy: 0.7374 - val_loss: 0.5203 - val_binary_accuracy: 0.7538
Epoch 3/100
338/338 [==============================] - 3s 9ms/step - loss: 0.5050 - binary_accuracy: 0.7659 - val_loss: 0.4969 - val_binary_accuracy: 0.7665
Epoch 4/100
338/338 [==============================] - 3s 9ms/step - loss: 0.4826 - binary_accuracy: 0.7787 - val_loss: 0.4824 - val_binary_accuracy: 0.7737
Epoch 5/100
338/338 [==============================] - 4s 10ms/step - loss: 0.4666 - binary_accuracy: 0.7854 - val_loss: 0.4725 - val_binary_accuracy: 0.7782
Epoch 6/100
338/338 [==============================] - 4s 10ms/step - loss: 0.4534 - binary_accuracy: 0.7923 - val_loss: 0.4655 - val_binary_accuracy: 0.7815
Epoch 7/100
338/338 [==============================] - 

Result: adding reward description further increased model performance

#### d) Best-Found Model: LR Text

**Preprocessing:**
- stopwords removed
- all text features included
- 2-grams, max_features=75000
- TFIDF Vectorizer + sublinear_tf 

**Model**:
- Activation = sigmoid (necessary to output probability between 0 and 1)
- GlorotNormal initilization
- RMSProp optimizer
- Batch Size: 512